<a href="https://colab.research.google.com/github/n1lays1ngh/Delhi-Air-Quality-Prediction/blob/main/Integrating_Weather_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
data_folder_path = '/content/drive/MyDrive/Pollutant Data'


In [3]:
weather_file_name_1 = 'weather_2013_to_2024_Nov.csv'
weather_file_name_2 = 'weather_2024_Dec_to_2025.csv'

In [5]:
cols_file_1 = [
    'DATE',
    'temp',
    'humidity',
    'precip',
    'windspeed',
    'sealevelpressure',
    'conditions'
]

In [6]:
cols_file_2 = [
    'datetime',
    'temp',
    'humidity',
    'precip',
    'windspeed',
    'sealevelpressure',
    'conditions'
]

In [7]:
rename_map_file_2 = {
    'datetime': 'DATE'
}

In [8]:
numeric_weather_cols = ['temp', 'humidity', 'precip', 'windspeed', 'sealevelpressure']

In [9]:
aqi_input_file = os.path.join(data_folder_path, 'delhi_aqi_feature_engineered.csv')
weather_input_file_1 = os.path.join(data_folder_path, weather_file_name_1)
weather_input_file_2 = os.path.join(data_folder_path, weather_file_name_2)
final_output_file = os.path.join(data_folder_path, 'delhi_aqi_final_dataset.csv')

In [11]:
df_weather_1 = pd.read_csv(weather_input_file_1, usecols=cols_file_1)

In [13]:
df_weather_1['Date'] = pd.to_datetime(df_weather_1['DATE'])

In [15]:
df_weather_1 = df_weather_1.drop('DATE', axis=1)

In [16]:
df_weather_1.head()

,temp,humidity,precip,windspeed,sealevelpressure,conditions,Date
0,28.7,39.7,2.0,22.3,1008.3,"Rain, Partially cloudy",2013-04-14
1,28.6,41.7,0.0,17.6,1005.1,Clear,2013-04-15
2,31.7,30.7,0.0,27.7,1001.7,Partially cloudy,2013-04-16
3,29.9,27.4,0.0,22.3,1002.5,Clear,2013-04-17
4,30.6,23.7,0.0,12.4,1003.4,Clear,2013-04-18


In [18]:
df_weather_2 = pd.read_csv(weather_input_file_2, usecols=cols_file_2)


In [20]:
df_weather_2 = df_weather_2.rename(columns=rename_map_file_2)

In [25]:
df_weather_2.head()

,temp,humidity,precip,windspeed,sealevelpressure,conditions,Date
0,18.6,64.2,0.0,14.8,1013.8,Clear,2024-12-01
1,18.5,68.2,0.0,9.4,1013.9,Clear,2024-12-02
2,18.7,67.0,0.0,14.8,1015.8,Clear,2024-12-03
3,18.6,58.8,0.0,18.4,1015.2,Clear,2024-12-04
4,16.6,52.3,0.0,21.9,1014.8,Clear,2024-12-05


In [22]:
df_weather_2['Date'] = pd.to_datetime(df_weather_2['DATE'])

In [24]:
df_weather_2 = df_weather_2.drop('DATE', axis=1)

In [26]:
df_weather = pd.concat([df_weather_1, df_weather_2], ignore_index=True)

In [29]:
df_weather = df_weather.drop_duplicates(subset=['Date'], keep='first')

,temp,humidity,precip,windspeed,sealevelpressure,conditions,Date
0,28.7,39.7,2.0,22.3,1008.3,"Rain, Partially cloudy",2013-04-14
1,28.6,41.7,0.0,17.6,1005.1,Clear,2013-04-15
2,31.7,30.7,0.0,27.7,1001.7,Partially cloudy,2013-04-16
3,29.9,27.4,0.0,22.3,1002.5,Clear,2013-04-17
4,30.6,23.7,0.0,12.4,1003.4,Clear,2013-04-18
...,...,...,...,...,...,...,...
3663,22.7,59.6,0.0,17.8,1013.2,Clear,2025-03-17
3664,22.6,57.9,0.0,17.3,1012.8,Clear,2025-03-18
3665,23.9,50.0,0.0,18.4,1013.6,Clear,2025-03-19
3666,25.1,47.9,0.0,20.5,1014.6,Clear,2025-03-20


In [46]:
df = pd.read_csv(aqi_input_file, parse_dates=['Date'])

In [48]:
df.shape

(3005, 22)

In [49]:
df_weather.shape

(3677, 7)

In [50]:
df_final = pd.merge(df, df_weather, on='Date', how='left')

In [51]:
df_final = df_final.dropna(subset=['AQI', 'AQI_lag_1'])

In [52]:
df_final.shape

(3005, 28)

In [53]:
null = df_final['temp'].isnull().sum()
print(null)

140


In [54]:
df_final = df_final.set_index('Date')
df_final['Month'] = df_final.index.month

In [55]:
for col in numeric_weather_cols:

    df_final[f'{col}_is_missing'] = df_final[col].isna().astype(int)

    seasonal_median = df_final.groupby('Month')[col].transform('median')
    df_final[col] = df_final[col].fillna(seasonal_median)

    df_final[col] = df_final[col].fillna(df_final[col].median())

In [56]:
df_final['conditions'] = df_final['conditions'].fillna('Unknown')

In [57]:
df_final.head()

,index,AQI,Month_sin,Month_cos,DayOfWeek_sin,DayOfWeek_cos,DayOfYear_sin,DayOfYear_cos,AQI_lag_1,AQI_lag_3,...,precip,windspeed,sealevelpressure,conditions,Month,temp_is_missing,humidity_is_missing,precip_is_missing,windspeed_is_missing,sealevelpressure_is_missing
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-08,7,288.0,0.5,0.866025,-0.781831,0.623490,0.137185,0.990545,278.0,321.0,...,0.0,13.0,1017.2,Unknown,1,1,1,1,1,1
2017-01-09,8,308.0,0.5,0.866025,0.000000,1.000000,0.154204,0.988039,288.0,351.0,...,0.0,13.0,1017.2,Unknown,1,1,1,1,1,1
2017-01-10,9,247.0,0.5,0.866025,0.781831,0.623490,0.171177,0.985240,308.0,278.0,...,0.0,13.0,1017.2,Unknown,1,1,1,1,1,1
2017-01-11,10,280.0,0.5,0.866025,0.974928,-0.222521,0.188099,0.982150,247.0,288.0,...,0.0,13.0,1017.2,Unknown,1,1,1,1,1,1
2017-01-12,11,323.0,0.5,0.866025,0.433884,-0.900969,0.204966,0.978769,280.0,308.0,...,0.0,13.0,1017.2,Unknown,1,1,1,1,1,1


In [58]:
all_conditions = df_final['conditions'].unique()
print(f"Found {len(all_conditions)} unique weather conditions.")

Found 6 unique weather conditions.


In [59]:
for cond in all_conditions:
    # Clean up condition names to make valid column names
    clean_cond_name = str(cond).replace(', ', '_').replace(' ', '_').lower()
    df_final[f'cond_{clean_cond_name}'] = (df_final['conditions'] == cond).astype(int)

In [61]:
df_final.head()

,index,AQI,Month_sin,Month_cos,DayOfWeek_sin,DayOfWeek_cos,DayOfYear_sin,DayOfYear_cos,AQI_lag_1,AQI_lag_3,...,humidity_is_missing,precip_is_missing,windspeed_is_missing,sealevelpressure_is_missing,cond_unknown,cond_clear,cond_partially_cloudy,cond_rain_partially_cloudy,cond_rain,cond_rain_overcast
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-08,7,288.0,0.5,0.866025,-0.781831,0.623490,0.137185,0.990545,278.0,321.0,...,1,1,1,1,1,0,0,0,0,0
2017-01-09,8,308.0,0.5,0.866025,0.000000,1.000000,0.154204,0.988039,288.0,351.0,...,1,1,1,1,1,0,0,0,0,0
2017-01-10,9,247.0,0.5,0.866025,0.781831,0.623490,0.171177,0.985240,308.0,278.0,...,1,1,1,1,1,0,0,0,0,0
2017-01-11,10,280.0,0.5,0.866025,0.974928,-0.222521,0.188099,0.982150,247.0,288.0,...,1,1,1,1,1,0,0,0,0,0
2017-01-12,11,323.0,0.5,0.866025,0.433884,-0.900969,0.204966,0.978769,280.0,308.0,...,1,1,1,1,1,0,0,0,0,0


In [62]:
df_final = df_final.drop(['Month', 'conditions'], axis=1)

In [64]:
df_final = df_final.reset_index()

In [65]:
df_final.to_csv(final_output_file, index=False)

In [66]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005 entries, 0 to 3004
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         3005 non-null   datetime64[ns]
 1   index                        3005 non-null   int64         
 2   AQI                          3005 non-null   float64       
 3   Month_sin                    3005 non-null   float64       
 4   Month_cos                    3005 non-null   float64       
 5   DayOfWeek_sin                3005 non-null   float64       
 6   DayOfWeek_cos                3005 non-null   float64       
 7   DayOfYear_sin                3005 non-null   float64       
 8   DayOfYear_cos                3005 non-null   float64       
 9   AQI_lag_1                    3005 non-null   float64       
 10  AQI_lag_3                    3005 non-null   float64       
 11  AQI_lag_7                    3005 non-null 